In [3]:
import numpy as np
np.random.seed(1234)
import pandas as pd


In [4]:
import keras

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\users\bodhv\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\bodhv\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\bodhv\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\bodhv\appdata\local\programs\python\python38\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "c:\users\bodhv\appdata\local\programs\python\python38\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load fai

In [61]:
# Keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras import backend as K


In [7]:
#plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [42]:
def read_dataset(filename):
    dataset = pd.read_csv(filename)#training_set_with_cue.csv
    return dataset
def get_data_labels(df):
    label = df['flow_size']
    del df['flow_size']
    return df, label


train_df = read_dataset('KMeans_training.csv')
test_df = read_dataset('KMeans_test.csv')
val_x = np.loadtxt('KMeans_validation.csv', delimiter = ',' , skiprows = 1, usecols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13])
val_y = np.loadtxt('KMeans_validation.csv', delimiter = ',' , skiprows = 1, usecols = 14)

train_x, train_y = get_data_labels(train_df)
test_x, test_y = get_data_labels(test_df)
val = (val_x, val_y)

In [39]:
val[0]

array([7.2000e+03, 7.2000e+03, 2.7180e+03, 1.0400e+02, 2.7180e+03,
       1.0400e+02, 5.6000e+01, 4.0000e+00, 7.2179e+00, 4.8040e-01,
       2.0800e+02, 1.1415e+04, 6.3910e+03, 2.7180e+03, 2.7180e+03])

# Neural Network

In [77]:
from keras import optimizers
adam = optimizers.Adam(lr=0.01)

In [65]:
def r2_score(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


#from https://stackoverflow.com/questions/45250100/kerasregressor-coefficient-of-determination-r2-score

In [87]:
model = Sequential()
model.add(Dense(3, input_shape = (14,)))
model.add(Dense(5, activation='relu'))
#model.add(Dense(5, activation='softmax'))
model.add(Dense(1, activation='relu'))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 3)                 45        
_________________________________________________________________
dense_53 (Dense)             (None, 5)                 20        
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 6         
Total params: 71
Trainable params: 71
Non-trainable params: 0
_________________________________________________________________


In [88]:

model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=[r2_score])
model.fit(train_x, train_y, epochs=20, validation_data = val, batch_size=10, shuffle = True)


#error metric
model.evaluate(test_x, test_y)

Train on 64169 samples, validate on 11839 samples
Epoch 1/20
64169/64169 [==============================] - 5s 76us/step - loss: 551135479453855.3125 - r2_score: -7295.2939 - val_loss: 568488413731362.1250 - val_r2_score: -0.8701
Epoch 2/20
64169/64169 [==============================] - 5s 75us/step - loss: 548148615477198.9375 - r2_score: -0.6731 - val_loss: 568488413731362.1250 - val_r2_score: -0.8701
Epoch 3/20
64169/64169 [==============================] - 5s 77us/step - loss: 548148615201636.9375 - r2_score: -0.6859 - val_loss: 568488413731362.1250 - val_r2_score: -0.8701
Epoch 4/20
64169/64169 [==============================] - 5s 76us/step - loss: 548148616650739.5000 - r2_score: -0.6751 - val_loss: 568488413731362.1250 - val_r2_score: -0.8701
Epoch 5/20
64169/64169 [==============================] - 5s 82us/step - loss: 548148613388916.5625 - r2_score: -0.6766 - val_loss: 568488413731362.1250 - val_r2_score: -0.8701
Epoch 6/20
64169/64169 [==============================] - 5s 8

[527127170433095.56, -0.4522804617881775]

In [47]:
model.evaluate(test_x, test_y)

15730/15730 [==============================] - 0s 25us/step


[527127170433095.56, 0.0]

In [69]:
predy = model.predict(test_x)

In [73]:
for i in range(len(predy)):
    print(test_y[i], "\t", predy[i])

1576 	 [3535.1096]
2725 	 [3570.9575]
3211 	 [3523.2317]
1604 	 [0.]
1494 	 [0.]
2997 	 [1785.1022]
2866 	 [0.]
3246 	 [0.]
22192 	 [20829.68]
1576 	 [0.]
112 	 [0.]
348 	 [0.]
1609 	 [0.]
525 	 [0.]
1525 	 [0.]
15238 	 [21316.152]
413 	 [0.]
60 	 [0.]
104 	 [0.]
22705 	 [26391.838]
1957 	 [0.]
2180 	 [0.]
9736 	 [0.]
156 	 [0.]
766152 	 [0.]
1576 	 [0.]
104 	 [0.]
52 	 [0.]
52 	 [0.]
194171 	 [0.]
21756 	 [0.]
4756 	 [0.]
1515 	 [0.]
3164 	 [0.]
104 	 [0.]
4669 	 [0.]
135469 	 [0.]
4578 	 [0.]
1588 	 [0.]
1515 	 [0.]
3103 	 [0.]
156 	 [0.]
156 	 [0.]
39914 	 [0.]
9547 	 [0.]
1640 	 [0.]
4526 	 [0.]
52 	 [0.]
55 	 [0.]
7720 	 [0.]
1463 	 [0.]
6341 	 [0.]
4669 	 [0.]
24165 	 [0.]
525 	 [0.]
104 	 [0.]
1473 	 [0.]
3962 	 [0.]
28713 	 [0.]
52 	 [0.]
156 	 [0.]
25960 	 [0.]
104 	 [0.]
15330 	 [0.]
1679 	 [0.]
361 	 [0.]
1731 	 [0.]
52 	 [0.]
39235 	 [0.]
52 	 [0.]
52 	 [0.]
156 	 [0.]
37038 	 [0.]
361 	 [0.]
24068 	 [0.]
7750 	 [0.]
1576 	 [0.]
40594 	 [0.]
361 	 [0.]
20189 	 [0.]
1464 	 [

5258 	 [0.]
5474 	 [0.]
5214 	 [0.]
37941 	 [0.]
6416 	 [0.]
4458 	 [0.]
65211 	 [0.]
6000 	 [0.]
3926 	 [0.]
9086 	 [0.]
14555 	 [0.]
13629 	 [0.]
4514 	 [0.]
6308 	 [0.]
65521 	 [0.]
4410 	 [0.]
5620 	 [0.]
25486 	 [0.]
3257 	 [0.]
3744 	 [0.]
2160 	 [0.]
122071418 	 [0.]
1576 	 [0.]
20031 	 [0.]
177078110 	 [0.]
1410 	 [0.]
4118 	 [0.]
1337 	 [0.]
4066 	 [0.]
5528 	 [0.]
1462 	 [0.]
4066 	 [0.]
6345 	 [0.]
21254 	 [0.]
28465 	 [0.]
362 	 [0.]
3829 	 [0.]
27727 	 [0.]
104 	 [0.]
104 	 [0.]
104 	 [0.]
312 	 [0.]
104 	 [0.]
104 	 [0.]
1576 	 [5084.876]
2725 	 [3986.9683]
112 	 [0.]
3099 	 [2235.5564]
1604 	 [0.]
1494 	 [0.]
2997 	 [0.]
2646 	 [0.]
3194 	 [0.]
22192 	 [17562.191]
356 	 [0.]
1557 	 [0.]
525 	 [0.]
1473 	 [0.]
13713 	 [12148.349]
112 	 [0.]
517 	 [0.]
1525 	 [0.]
2312 	 [0.]
1576 	 [0.]
112 	 [0.]
878 	 [0.]
318 	 [0.]
1068842 	 [0.]
20241 	 [0.]
20137 	 [0.]
1515 	 [0.]
1463 	 [0.]
4756 	 [0.]
104 	 [0.]
260 	 [0.]
52 	 [0.]
818883 	 [0.]
18110 	 [0.]
1576 	 [0.]
3063 	 

4392 	 [0.]
108966 	 [0.]
14124 	 [0.]
22646 	 [0.]
18267 	 [0.]
4545 	 [0.]
2842 	 [0.]
6166 	 [0.]
95966 	 [0.]
13032 	 [0.]
3872 	 [0.]
3710 	 [0.]
40670 	 [0.]
4545 	 [0.]
78524 	 [0.]
4496 	 [0.]
5958 	 [0.]
4548 	 [0.]
14072 	 [0.]
25330 	 [0.]
3265 	 [0.]
3752 	 [0.]
20083 	 [0.]
2168 	 [0.]
442280758 	 [0.]
52 	 [0.]
1576 	 [0.]
6892 	 [0.]
52 	 [0.]
1576 	 [0.]
1462 	 [0.]
13690 	 [0.]
1462 	 [0.]
5528 	 [0.]
52 	 [0.]
28936 	 [0.]
1083 	 [0.]
1083 	 [0.]
11302 	 [0.]
23792 	 [0.]
5508 	 [0.]
208 	 [0.]
208 	 [0.]
208 	 [0.]
104 	 [0.]
364 	 [0.]
208 	 [0.]
1576 	 [0.]
2725 	 [864.1819]
4659 	 [4447.534]
104 	 [0.]
1494 	 [0.]
30717 	 [36258.727]
356 	 [0.]
3560 	 [0.]
13585 	 [11194.403]
577 	 [0.]
1473 	 [0.]
2009 	 [0.]
3616884 	 [0.]
20137 	 [0.]
1463 	 [0.]
52 	 [0.]
1588 	 [0.]
1576 	 [0.]
20189 	 [0.]
2999 	 [0.]
4721 	 [0.]
1515 	 [0.]
4669 	 [0.]
4379 	 [0.]
2926 	 [0.]
4756 	 [0.]
52 	 [0.]
7461 	 [0.]
6132 	 [0.]
4669 	 [0.]
1576 	 [0.]
2999 	 [0.]
6341 	 [0.]
5114 

1457 	 [0.]
19654 	 [0.]
1195 	 [0.]
1195 	 [0.]
7375 	 [0.]
32149 	 [0.]
9098 	 [0.]
4622 	 [0.]
13866 	 [0.]
4236 	 [0.]
6216 	 [0.]
2793 	 [0.]
115836 	 [0.]
20070 	 [0.]
1195 	 [0.]
1195 	 [0.]
1195 	 [0.]
20622 	 [0.]
34975 	 [0.]
13647 	 [0.]
13647 	 [0.]
4636 	 [0.]
4441 	 [0.]
7580 	 [0.]
89480 	 [0.]
19030 	 [0.]
3768 	 [0.]
1576 	 [0.]
48622 	 [0.]
4549 	 [0.]
4549 	 [0.]
94915 	 [0.]
7576 	 [0.]
5126 	 [0.]
5542 	 [0.]
18606 	 [0.]
25486 	 [0.]
3325 	 [0.]
5824 	 [0.]
259906022 	 [0.]
20083 	 [0.]
52 	 [0.]
52 	 [0.]
20031 	 [0.]
90755546 	 [0.]
1410 	 [0.]
2826 	 [0.]
1410 	 [0.]
4883 	 [0.]
1576 	 [0.]
19713 	 [0.]
52 	 [0.]
1462 	 [0.]
5476 	 [0.]
70759578 	 [0.]
78455388 	 [0.]
141083345 	 [0.]
52 	 [0.]
1462 	 [0.]
4153 	 [0.]
1576 	 [0.]
11056 	 [0.]
28624 	 [0.]
27386 	 [0.]
8476 	 [0.]
7658 	 [0.]
20060 	 [0.]
260 	 [0.]
208 	 [0.]
208 	 [0.]
208 	 [0.]
104 	 [0.]
1576 	 [0.]
2777 	 [1030.8596]
6339 	 [7911.2275]
1494 	 [0.]
31021 	 [30555.543]
3560 	 [0.]
15632 	 [1

7421 	 [0.]
34148 	 [0.]
9190 	 [0.]
4682 	 [0.]
186812 	 [0.]
10534 	 [0.]
7352 	 [0.]
11532 	 [0.]
30539 	 [0.]
7494 	 [0.]
25231 	 [0.]
18526 	 [0.]
9190 	 [0.]
7248 	 [0.]
193956 	 [0.]
7612 	 [0.]
10430 	 [0.]
30697 	 [0.]
7421 	 [0.]
7421 	 [0.]
30396 	 [0.]
4595 	 [0.]
1576 	 [0.]
4595 	 [0.]
4595 	 [0.]
170677 	 [0.]
7560 	 [0.]
11366 	 [0.]
6988 	 [0.]
29771 	 [0.]
7421 	 [0.]
19437 	 [0.]
19306 	 [0.]
4595 	 [0.]
9336 	 [0.]
144438 	 [0.]
11470 	 [0.]
7300 	 [0.]
6884 	 [0.]
25538 	 [0.]
3257 	 [0.]
3744 	 [0.]
318054767 	 [0.]
52 	 [0.]
2112 	 [0.]
2826 	 [0.]
4066 	 [0.]
19713 	 [0.]
1410 	 [0.]
1462 	 [0.]
19713 	 [0.]
6840 	 [0.]
4236 	 [0.]
19587498 	 [0.]
4066 	 [0.]
1462 	 [0.]
137437344 	 [0.]
13320 	 [0.]
8219 	 [0.]
8954 	 [0.]
1462 	 [0.]
4066 	 [0.]
1462 	 [0.]
587 	 [0.]
4066 	 [0.]
28884 	 [0.]
27438 	 [0.]
6655 	 [0.]
7473 	 [0.]
18142 	 [0.]
7658 	 [0.]
7658 	 [0.]
3916 	 [0.]
104 	 [0.]
104 	 [0.]
104 	 [0.]
312 	 [0.]
104 	 [0.]
1576 	 [527.3444]
2725 	 [112

...

21263 	 [0.]
23405 	 [0.]
8641 	 [0.]
27043 	 [0.]
525 	 [0.]
1473 	 [0.]
31025 	 [0.]
7281 	 [0.]
7281 	 [0.]
7354 	 [0.]
7441 	 [0.]
11882 	 [0.]
4455 	 [0.]
4528 	 [0.]
4528 	 [0.]
9056 	 [0.]
4455 	 [0.]
4455 	 [0.]
208184 	 [0.]
16516 	 [0.]
11488 	 [0.]
14412 	 [0.]
28780 	 [0.]
29771 	 [0.]
35464 	 [0.]
28446 	 [0.]
13872 	 [0.]
9190 	 [0.]
9190 	 [0.]
193265 	 [0.]
11398 	 [0.]
8760 	 [0.]
8812 	 [0.]
32080 	 [0.]
14842 	 [0.]
10655 	 [0.]
23083 	 [0.]
9277 	 [0.]
13959 	 [0.]
4595 	 [0.]
4595 	 [0.]
4682 	 [0.]
13166 	 [0.]
174371 	 [0.]
10637 	 [0.]
10639 	 [0.]
30796 	 [0.]
14842 	 [0.]
28057 	 [0.]
9190 	 [0.]
13785 	 [0.]
4595 	 [0.]
193177 	 [0.]
12906 	 [0.]
7616 	 [0.]
7668 	 [0.]
29684 	 [0.]
7421 	 [0.]
7421 	 [0.]
23083 	 [0.]
14605 	 [0.]
4595 	 [0.]
9190 	 [0.]
9190 	 [0.]
13062 	 [0.]
172370 	 [0.]
6940 	 [0.]
7928 	 [0.]
30452 	 [0.]
14842 	 [0.]
7421 	 [0.]
24939 	 [0.]
14004 	 [0.]
4668 	 [0.]
202015 	 [0.]
8500 	 [0.]
8500 	 [0.]
11762 	 [0.]
29684 	 [0.]
1484

1484 	 [0.]
4669 	 [0.]
1576 	 [0.]
1411 	 [0.]
1463 	 [0.]
15821 	 [0.]
2999 	 [0.]
3051 	 [0.]
10888 	 [0.]
2999 	 [0.]
2999 	 [0.]
1463 	 [0.]
6341 	 [0.]
14015 	 [0.]
1576 	 [0.]
4462 	 [0.]
4669 	 [0.]
4744 	 [0.]
2998 	 [0.]
1462 	 [0.]
4756 	 [0.]
52 	 [0.]
52 	 [0.]
1576 	 [0.]
1606 	 [0.]
25848 	 [0.]
2943 	 [0.]
38628 	 [0.]
7750 	 [0.]
40960 	 [0.]
3082 	 [0.]
486 	 [0.]
1731 	 [0.]
54173 	 [0.]
4836 	 [0.]
12180 	 [0.]
27369 	 [0.]
3230 	 [0.]
38358 	 [0.]
8109 	 [0.]
40960 	 [0.]
57181 	 [0.]
12036 	 [0.]
1606 	 [0.]
27525 	 [0.]
3312 	 [0.]
38001 	 [0.]
8343 	 [0.]
2298 	 [0.]
29090 	 [0.]
4785 	 [0.]
4785 	 [0.]
4897 	 [0.]
7318 	 [0.]
8051 	 [0.]
27250 	 [0.]
4405 	 [0.]
4405 	 [0.]
27145 	 [0.]
16047 	 [0.]
7908 	 [0.]
2508 	 [0.]
5816 	 [0.]
12434 	 [0.]
10181 	 [0.]
154425 	 [0.]
8385 	 [0.]
43060 	 [0.]
9090 	 [0.]
7276 	 [0.]
8507 	 [0.]
6922 	 [0.]
26885 	 [0.]
8758 	 [0.]
70980 	 [0.]
1194 	 [0.]
42938 	 [0.]
9125 	 [0.]
9981 	 [0.]
12498 	 [0.]
7362 	 [0.]
26625

9166 	 [0.]
5796 	 [0.]
7036 	 [0.]
87636 	 [0.]
7200 	 [0.]
20412 	 [0.]
29636 	 [0.]
14818 	 [0.]
35145 	 [0.]
18332 	 [0.]
4583 	 [0.]
9166 	 [0.]
6004 	 [0.]
121314 	 [0.]
7356 	 [0.]
22914 	 [0.]
7192 	 [0.]
7409 	 [0.]
7409 	 [0.]
7409 	 [0.]
29584 	 [0.]
24135 	 [0.]
4656 	 [0.]
9312 	 [0.]
125211 	 [0.]
24321 	 [0.]
4711 	 [0.]
5407 	 [0.]
5569 	 [0.]
20622 	 [0.]
30456 	 [0.]
7496 	 [0.]
7409 	 [0.]
7409 	 [0.]
27630 	 [0.]
18332 	 [0.]
4670 	 [0.]
56677 	 [0.]
5796 	 [0.]
5892 	 [0.]
20782 	 [0.]
5588 	 [0.]
40718 	 [0.]
44196 	 [0.]
7461 	 [0.]
7461 	 [0.]
22227 	 [0.]
21978 	 [0.]
9253 	 [0.]
9253 	 [0.]
112329 	 [0.]
21874 	 [0.]
6212 	 [0.]
6464 	 [0.]
7148 	 [0.]
29636 	 [0.]
7409 	 [0.]
7409 	 [0.]
44398 	 [0.]
9312 	 [0.]
4656 	 [0.]
104360 	 [0.]
4035 	 [0.]
6784 	 [0.]
20626 	 [0.]
6152 	 [0.]
1761 	 [0.]
3309 	 [0.]
3796 	 [0.]
419906670 	 [0.]
13157 	 [0.]
4066 	 [0.]
1410 	 [0.]
4066 	 [0.]
20031 	 [0.]
1462 	 [0.]
13157 	 [0.]
1462 	 [0.]
2774 	 [0.]
211 	 [0.]
4

17543 	 [0.]
4597 	 [0.]
6173 	 [0.]
4597 	 [0.]
13878 	 [0.]
21806 	 [0.]
17424 	 [0.]
144072 	 [0.]
27388 	 [0.]
25486 	 [0.]
26972 	 [0.]
3257 	 [0.]
20031 	 [0.]
6000 	 [0.]
515554713 	 [0.]
1462 	 [0.]
6892 	 [0.]
52 	 [0.]
104 	 [0.]
6892 	 [0.]
1576 	 [0.]
2826 	 [0.]
2774 	 [0.]
1462 	 [0.]
6944 	 [0.]
20031 	 [0.]
5528 	 [0.]
4066 	 [0.]
1410 	 [0.]
1462 	 [0.]
4744 	 [0.]
1071 	 [0.]
1462 	 [0.]
52 	 [0.]
4153 	 [0.]
28832 	 [0.]
27525 	 [0.]
6655 	 [0.]
6655 	 [0.]
26707 	 [0.]
12305 	 [0.]
3829 	 [0.]
11487 	 [0.]
4089 	 [0.]
156 	 [0.]
156 	 [0.]
104 	 [0.]
104 	 [0.]
1576 	 [0.]
1576 	 [589.3598]
2777 	 [1113.9022]
4763 	 [3227.773]
1494 	 [0.]
31073 	 [29658.533]
4992 	 [0.]
15372 	 [14800.445]
22869 	 [32387.33]
22869 	 [22959.615]
12625 	 [0.]
3739708 	 [0.]
1576 	 [0.]
52 	 [0.]
2999 	 [0.]
24881 	 [0.]
13320 	 [0.]
2999 	 [0.]
1463 	 [0.]
4669 	 [0.]
1576 	 [0.]
8997 	 [0.]
18344 	 [0.]
2998 	 [0.]
6131 	 [0.]
25900 	 [0.]
2943 	 [0.]
32887 	 [0.]
52 	 [0.]
41064 	 [

31381 	 [0.]
27494 	 [0.]
167064 	 [0.]
7436 	 [0.]
10998 	 [0.]
7176 	 [0.]
29668 	 [0.]
15760 	 [0.]
25781 	 [0.]
18312 	 [0.]
11466 	 [0.]
209678 	 [0.]
7956 	 [0.]
7488 	 [0.]
15654 	 [0.]
14730 	 [0.]
59945 	 [0.]
7436 	 [0.]
216684 	 [0.]
7436 	 [0.]
10894 	 [0.]
30594 	 [0.]
22251 	 [0.]
4591 	 [0.]
32311 	 [0.]
130014 	 [0.]
7904 	 [0.]
11726 	 [0.]
7983 	 [0.]
25538 	 [0.]
3273 	 [0.]
3812 	 [0.]
398861288 	 [0.]
1462 	 [0.]
2826 	 [0.]
4066 	 [0.]
1576 	 [0.]
1462 	 [0.]
6504 	 [0.]
1576 	 [0.]
5528 	 [0.]
1462 	 [0.]
4066 	 [0.]
1462 	 [0.]
5528 	 [0.]
1462 	 [0.]
11085 	 [0.]
5528 	 [0.]
1576 	 [0.]
5528 	 [0.]
3954344 	 [0.]
3345664 	 [0.]
5528 	 [0.]
1325948 	 [0.]
214328 	 [0.]
998504 	 [0.]
3681200 	 [0.]
2541776 	 [0.]
1791589 	 [0.]
3298834 	 [0.]
4118 	 [0.]
1606 	 [0.]
28780 	 [0.]
27438 	 [0.]
24560 	 [0.]
26751 	 [0.]
156 	 [0.]
104 	 [0.]
312 	 [0.]
52 	 [0.]
104 	 [0.]
104 	 [0.]
1576 	 [3510.374]
2777 	 [2724.8386]
112 	 [0.]
1498 	 [266.60522]
1601 	 [0.]
1604